# Swarm Simulator for Outer Space

This swarm simulator will explore different swarm algorithms for an application in outer space. This approach will divide a plot into grid squares, which can then be analysed to determine and control the behaviour of each individual agent in the swarm.

In [26]:
#imports
import sympy as sym
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math

from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import deque
from scipy.spatial.distance import cdist, pdist, euclidean
from sympy import sin, cos, symbols, lambdify

from robot import Robot
from dust_devil import DustDevil
from live_functions_tracking import update_decay, initialise,random_position, grid_center,positions,bounce,magnitude,unit,division_check,physics_walk,dust_check,update_timestep,cluster_function,G_transition,dist,random_walk,dust,trajectory_dust,update_dust,load_positions,pre_initialise

import random
import datetime
import os
import math
import sys

import Processing_Functions_Tracking


import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import json
import pickle

from ast import literal_eval

app = dash.Dash(__name__)

In [27]:
x = np.array([1,2,3,4,5,6,7,8,9,10])
y = np.array([2,4,6,8,10,12,14,16,18,20])
graph_end = Processing_Functions_Tracking.performance_graph(y,x,1,"Test","Testing X","Testing Y")
graph_end.write_image("..\Experiments\Testing_Graph\Testing4.png")

Test


In [28]:
print("Kernel Check")

Kernel Check


In [29]:
#Plotly offline mode
init_notebook_mode(connected=True)

In [30]:
#function to calculate area coverage using grid metric
def grid_coverage(grid,positions,length):
    #creating the grid bins for area coverage
    x_bins = np.linspace(grid[0],grid[1],length+1)
    y_bins = np.linspace(grid[0],grid[1],length+1)
    
    x_positions = positions[0]
    y_positions = positions[1]
    #initialising empty coordinates array for use in the grid metric
    coordinates = []
    for i in range(len(x_positions)):
        #setting current position
        x_current = x_positions[i]
        y_current = y_positions[i]
        if(grid[0]<=x_current<=grid[1] and grid[0]<=y_current<=grid[1]):
            #searching for the x anf y coordinates using the numpy search sorted function combined with the previously defined bins
            x = np.searchsorted(x_bins, x_current, side ='left')
            y = np.searchsorted(y_bins, y_current, side ='left')
            coordinates.append((x,y))

            print(coordinates)


    rm_duplicates = list(set(coordinates)) #set is used to remove duplicates, as sets require distinct elements
    area_coverage = len(rm_duplicates) #converting it to a percentage
    total = length*length
    return area_coverage

In [31]:
def check_distance(robot,R,distance_array, multiply,like):
    positions = []
    for displacement in distance_array:
        robot.update_counter()
        if(like):
            if(dispalcement > 0 and (displacement*(1/robot.lattice))<multiply*R):
                positions.append(displacement)
        else:
            if(displacement > 0 and displacement<multiply*R):
                positions.append(displacement)
    return np.array(positions)

In [32]:
"""positions = [[379.46846005717373, 438.02847717862016, 153.89596436963546, -193.1283557781956, -416.6839770913143, 34.081562711073445, 374.72181409793734, 4.2575140859884275, 544.4866167789752, 23.190850898954615, 206.292374293048, 459.30724154559175, -466.8672644979862, -150.38320663356518, -397.21668016133697, -371.0739121077903, 478.0788958724223, 282.45802392448843, -384.6534189059488, -144.80793054458428, -106.02592213527876, -444.43440341272503, -170.22569193162536, 382.92004456397865, 62.87070951518658, 212.95973786111648, -278.0567838144116, 115.1011207349581, -130.45332167322687, 205.04799777830556, 365.2264611175408, -23.515984683000326, 445.35388090506376, 98.53544732837173, 290.1076391910268, 202.11715479254326, 78.77032842771135, -42.89735025816067, -213.6628351057763, -35.500698734810456, 267.70466918435454, -485.64700248216207, 361.8527778542226, -296.71363260442513, -338.56952980783785, 165.3445238355776, 263.37547521526204, 359.43633016009665, 467.6646740557361, 104.77624573139161, 22.802832028931817, 52.73052471062435, -90.0057929627286, -98.59782691890629, -227.5188903050415, -573.2341830580037, -580.8326772744379, -13.063102460866759, 253.48036728075232, 161.01711053412822, -204.2595823910022, -480.59403217739134, 80.88401070149266, -608.1992177828363, 297.83433951064205, -316.71542444098327, 131.36911256856305, -497.0272951287137, -290.5273579759531, -409.8659901666948, 47.456347223082176, 323.3539728944321, -543.7684205850073, 379.78709845410674, 121.72832157016853, -209.01836032758393, 535.1602882987758, 549.207657699739, -254.10778066703574, -273.7172482391562, 48.06370774024232, -330.92367051605993, -54.73831221701292, 291.9691912442395, -47.0547013703526, 288.4613827402791, -312.1418887461871, 182.28356368866713, -398.5758594157063, -67.27356634825496, 0.031665638870082, -205.99973061025287, -509.33761911861745, 278.0730396276819, 189.7830598804298, 451.72116568297275, 198.535741465554, -121.00226946769848, -40.98085357848208, 337.6674334402037], [193.02754782914445, 454.5042113341391, -100.92137910004897, 42.73268937369279, -49.679767542452396, -410.48798824719745, 81.82807907994734, 445.55917342798534, 89.96929697234395, 250.0259556170741, 87.41833872075051, 133.3384328017171, -229.45480008921845, -505.01137623725106, 118.75792908404702, -262.13214566288497, 250.2075485728992, 436.6719202797577, 352.7205989049511, 209.42180023460577, 21.884694777627473, 40.25440129090552, 303.0328217899561, 289.9058830186664, 529.0254004423562, 316.7822320939215, -277.6194169251578, 76.21550582522042, -278.7533095286261, -366.26268424563386, -215.05528710142192, 81.30701753336464, -369.010993564116, 366.92015463401185, -60.17686820154037, -14.09842721581898, 169.2458360853655, -231.16839481272285, -109.10706887520573, 563.5289509758492, 238.49224250257612, 336.99743083799547, 384.98556185196446, 129.87691498490432, -165.3777398733705, 523.7653545787055, 537.7224160878344, -311.88547773144955, -65.77239858220321, -250.85317375837485, -309.74202514256876, -88.89297412647363, 475.213208992796, 375.2858493228242, -439.3506909932829, 163.32487662995197, -102.10636450206604, -552.9445461095431, -554.1950272403276, -531.5171372566158, 138.26975695917804, -124.34040953094612, -592.5956897313303, -10.72837940030393, -392.0804717921708, -71.05680996082148, -433.6177162693273, -325.55224682922926, 27.009085123259204, 219.32524148135457, -183.00076288226694, -483.74204352537174, 68.96170014446278, -17.259738974716335, 274.068202881296, -340.99064542325107, -223.6484491607289, -12.409889257920756, 422.77390658817546, 323.63748322858896, 12.491227031598967, -424.87210080149634, -460.97296804859513, 144.38365899621817, 182.50733406961103, 38.0908085787018, 230.96593987619093, 186.02118863703964, -359.63510863792385, -358.44498807936657, 347.51582237356797, -206.7621265707475, 240.1251116438812, -163.5550254008532, 409.48407753219686, -168.65554545819106, -271.3737154444477, -77.09110655116845, -135.18071874470147, 597.2627315131041]]
x_0 = positions[0]
y_0 = positions[1]
print((grid_coverage(np.array([-500,500]),np.array([x_0,y_0]),10)))"""


'positions = [[379.46846005717373, 438.02847717862016, 153.89596436963546, -193.1283557781956, -416.6839770913143, 34.081562711073445, 374.72181409793734, 4.2575140859884275, 544.4866167789752, 23.190850898954615, 206.292374293048, 459.30724154559175, -466.8672644979862, -150.38320663356518, -397.21668016133697, -371.0739121077903, 478.0788958724223, 282.45802392448843, -384.6534189059488, -144.80793054458428, -106.02592213527876, -444.43440341272503, -170.22569193162536, 382.92004456397865, 62.87070951518658, 212.95973786111648, -278.0567838144116, 115.1011207349581, -130.45332167322687, 205.04799777830556, 365.2264611175408, -23.515984683000326, 445.35388090506376, 98.53544732837173, 290.1076391910268, 202.11715479254326, 78.77032842771135, -42.89735025816067, -213.6628351057763, -35.500698734810456, 267.70466918435454, -485.64700248216207, 361.8527778542226, -296.71363260442513, -338.56952980783785, 165.3445238355776, 263.37547521526204, 359.43633016009665, 467.6646740557361, 104.77

In [33]:

#Function to call physics based area coverage algorithm
def physics_walk_honey(swarm,G,power,R,max_force,multiply,timestep,lattice_constants,velocity_coefficient,max_force_overall):
    '''
    Updates the robot position based on a physicomimetics algorithm

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None

    '''

    #print("Physics Called \n===============")
    #setting initial constants
    #setting initial constants
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    x,y = np.array(positions(swarm))
    type_0,type_1 = split(swarm)
    x_0,y_0 = np.array(positions(type_0))
    x_1,y_1 = np.array(positions(type_1))
    #print("Length 0:",len(x_0))
    #print("Length 1: ",len(x_1))
    

    #looping through each robot in the swarm
    for i in range(len(swarm)):
         

        force_0 = np.array([0,0])
        force_1 = np.array([0,0])
        #selecting robots one by one
        robot = swarm[i]
        
        robot.update_velocity(velocity_coefficient*velocity_x,velocity_coefficient*velocity_y)    
        #initialising the forces as 0
        force = np.array([0,0])
        
        R_0 = R
        R_1 = R
        #x_dist_unsorted,y_dist_unsorted = dist(robot,swarm)
        
        #finding the distance between the current robot and the other robots
        delta_x0,delta_y0 = dist(robot,x_0,y_0)
        #print([delta_x0,delta_y0])
        delta_x1,delta_y1 = dist(robot,x_1,y_1)
        """#finding the distance between the current robot and the other robots
        delta_x0,delta_y0 = dist(robot,x,y)
        delta_x1,delta_y1 = dist(robot,x,y)"""
        
        #calculating the distance from the robots to the current robot, split up according to robot type
        distance_0 = np.sqrt(np.square(delta_x0)+np.square(delta_y0))
        distance_1 = np.sqrt(np.square(delta_x1)+np.square(delta_y1))
        
        #finding the corresponding booleans
        multiply_like = 1.3
        multiply_unlike = 1.7
        
        constant = 1/robot.lattice
        scale = R/20
        if(robot.identifier == 0):
            R_0 = R
            R_1 = R
            distance_local_0 = np.logical_and(distance_0>0,(distance_0*constant)<multiply_like*R)
            distance_local_1 = np.logical_and(distance_1>0,distance_1<multiply_unlike*R)
            like_0 = True
            like_1 = False
            neighbourhood_0 = np.where(distance_local_0)
            #print("Raw Indices: ", neighbourhood_0)
            list_neighbours_0 = list(neighbourhood_0[0])
            neighbours_0 = [type_0[k] for k in list_neighbours_0]
            #print(list_neighbours_0)
            
            
            neighbourhood_1 = np.where(distance_local_1)
            #print("Raw Indices: ", neighbourhood_1)
            list_neighbours_1 = list(neighbourhood_1[0])
            neighbours_1 = [type_1[l] for l in list_neighbours_1]
            #print(list_neighbours_1)
            
        if(robot.identifier == 1):
            like_0 = False
            like_1 = True
            R_0 = R
            R_1 = R
            distance_local_0 = np.logical_and(distance_0>0,distance_0<multiply_unlike*R)
            distance_local_1 = np.logical_and(distance_1>0,(distance_1*constant)<multiply_like*R)
    
            neighbourhood_0 = np.where(distance_local_0)
            #print("Raw Indices: ", neighbourhood_0)
            list_neighbours_0 = list(neighbourhood_0[0])
            neighbours_0 = [type_0[m] for m in list_neighbours_0]
            #print(list_neighbours_0)
            
            neighbourhood_1 = np.where(distance_local_1)
            #print("Raw Indices: ", neighbourhood_1)
            list_neighbours_1 = list(neighbourhood_1[0])
            neighbours_1 = [type_1[n] for n in list_neighbours_1]
            #print(list_neighbours_1)
            
        #print(like_0)
        #print(like_1)
        
        x_dist_0 = delta_x0[distance_local_0]
        y_dist_0 = delta_y0[distance_local_0]    
        positions_0 = np.array([x_dist_0,y_dist_0])
        
        x_dist_1 = delta_x1[distance_local_1]
        y_dist_1 = delta_y1[distance_local_1]
        positions_1 = np.array([x_dist_1,y_dist_1])
        
        #print("Neighbourhood Distances 0: ", positions_0)
        #print("Neighbourhood Distances 1: ", positions_1)
        #print("Neighbourhood Distance Magnitude 0: ", np.sqrt(np.square(positions_0[0])+np.square(positions_0[1])))
        #print("Neighbourhood Distance Magnitude 1: ", np.sqrt(np.square(positions_1[0])+np.square(positions_1[1])))
        force_0 = artificial_gravity(neighbours_0,positions_0,robot,max_force,R_0,G,like_0,max_force_overall,scale)
        force_1 = artificial_gravity(neighbours_1,positions_1,robot,max_force,R_1,G,like_1,max_force_overall,scale)
        
        friction = random.uniform(-0.1,0.1)
        force = force_0 + force_1 #+ friction

        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx#velocity_coefficient*
        velocity_y = robot.y_velocity + delta_vy#velocity_coefficient*        
    
        #keeping the velocity within the maximum velocity of the robot
        if((velocity_x**2+velocity_y**2)>((robot.max_velocity)**2)):
            #calculating unit vector of the velocity
            unit_velocity = unit([velocity_x,velocity_y])
            
            #multiplying it by the maximum velocity
            velocity = unit_velocity*robot.max_velocity
            
            #setting the new velocity equal to the respective updated maximum velocity components
            velocity_x = velocity[0]
            velocity_y = velocity[1]
        
        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change
        
        #updating the position
        robot.update_position(x_updated,y_updated)
        robot.update_counter()
        #print("Finished 1 Loop within Physics Walk")
    #print("Finished the overall Physics Walk Calculation")

In [34]:
#Function to calculate artificial gravity forces
def artificial_gravity(neighbourhood,positions, robot, max_force,R,G,like,max_force_overall,scale):
        # print("R: ",R)
        # print("Constant: ", constant)
        force = np.array([0,0])
        force_change_dir = 0
        #print(neighbours)
        #print("Artificial Gravity Neighbours: ", neighbours)
        #print("Artificial Gravity Neighbours Magnitudes: ", (np.sqrt(np.square(neighbours[0])+np.square(neighbours[1]))))
        # print(R*1.7)
        loop_counter = 0
        
        for x,y in zip(positions[0],positions[1]):
            
                
            constant = 1/robot.lattice
                
            object_position = np.array([(neighbourhood[loop_counter].x-robot.x),(neighbourhood[loop_counter].y-robot.y)])
            
            #print("Object Distance: ", np.array([delta_x,delta_y]))
            #distance_0 = np.sqrt(np.square(delta_x)+np.square(delta_y))
            
            #neighbour = neighbours[i]
            #print(i)
            #storing current position 
            current_position = np.array([x,y])
            """if(not np.array_equal(current_position,object_position)): 
                print("Object Distance: ", object_position)
                print("Neighbourhood Distance: ", current_position)
            """
            #calculating magnitude of positions
            mag_unshifted = (magnitude(current_position[0],current_position[1]))
            """#print(mag_unshifted)
            if(mag_unshifted > R):
                #print("Larger Distance Value: ", mag_unshifted)
                if(mag_unshifted>1.7*R):
                    #print("Error! Distance is too large. Value is ",mag_unshifted)"""
            if(like):
                mag = mag_unshifted*constant
            else:
                mag = mag_unshifted

            
            #calculating the force
            numerator = (G*(robot.mass**2))
            denominator = mag**2

            
            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)
            
            #calculating the unit vector of the position
            distance_unit = unit(current_position)
            
            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>R):
                force_change_dir = distance_unit
            
            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])
       
            #calculating new force change based on direction and magnitude
            force_delta = (force_change_dir*force_change)/scale

            #constraining force to the maximum
            if((force_delta[0]**2+force_delta[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force_delta[0],force_delta[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force_delta[0] = updating_force[0]
                force_delta[1] = updating_force[1]
            force = force+force_delta
            loop_counter = loop_counter + 1
            
        #setting force maximum for sum of force contributions
        if((force[0]**2+force[1]**2)>((max_force_overall)**2)):
                #calculating unit vector of the force
                unit_force = unit([force[0],force[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force[0] = updating_force[0]
                force[1] = updating_force[1]
        #print("Finished")
        return force
    

In [35]:
#Function to calculate distance between a robot and the swarm
def dist(robot, x,y):
    #finding the distance between the current robot and the other robots
    x_dist = x-robot.x
    y_dist = y-robot.y
    return x_dist,y_dist

In [36]:
#Function to split swarm into different types
def split(swarm):
    type_0 = []
    type_1 = []
    for robot in swarm:
        if(robot.identifier == 0):
            type_0.append(robot)
        if(robot.identifier == 1):
            type_1.append(robot)

            
    return type_0,type_1
    

In [37]:
def broadcast(swarm, detection_list, set_R):
    #retrieving x and y positions
    x,y = positions(swarm)
   
    detecting = np.where(detection_list)
    list_detected = list(detecting[0])
    for i in list_detected:
        robot = swarm[i]
        x_copy,y_copy = x.copy(),y.copy()
        #finding the distance between the current robot and the other robots
        x_dist_unsorted = np.array(x_copy)-robot.x
        y_dist_unsorted = np.array(y_copy)-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))
       
        #determining the robots within 1.5R distance
        distance_local = np.logical_and(distance>0,distance<multiply*R)   
        
        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        
        neighbourhood = np.where(distance_local)
        list_neighbours = list(neighbourhood[0])

        neighbours = [swarm[i] for i in list_neighbours]
        #looping through the current robots neighbourhood
        for j in list_neighbours:
            swarm[j].R = set_R
    
            

In [38]:
#Function to calculate the G transition parameter
def G_square(max_force,R,power):
    '''
    Calculates the transition value between liquid and solid states for the G parameter

            Parameters:
                   max_force (float): the maximum float limit on the force
                   R (float): the chosen R seperation float value
                   power (float): the chosen power for the artificial gravity equation
                   
            Returns:
                   G_transition (float): the G transition value for the current swarm setup

    '''
    G_transition = (max_force*(R**power))/(2+2*math.sqrt(2))
    return G_transition

## Physics Constants

In [39]:
#setting physics constants
power = 2
R = 20
#max_force = 1

multiply = 1.5

In [40]:
#initialising parameters
conversion_ms = 1/3.6 #km/h to m/s conversion actor
timer = 0


#Robot Parameters
robot_speed = 2 #m/s
robot_radius = 1 
robot_mass = 1 #kgfro
x_robot = []
y_robot = []
min_neighbour = []
gen_random = 0 #the random angle

#dust devil parameters
dust_time = 170 #time in seconds the dust devil will survive 
dust_speed = 5 #m/s
dust_radius = 6
dust_devils = []
detection = 100

#Area Parameters
area = 2 #area in km^2 
side = 1 #each side length in m
per_square_km = 100/(60*60*24)
probability_dust = per_square_km*4#(per_square_km*area) #number of dust devils per second






In [ ]:
def simulation_run(solution,robot_number,max_force,seed,base_path):

    global grid_performance
    global fitness_results
    global performance_metric
    global interception_metric
    #preinitialise_path = "../Detecting Dust Devils - Passive Survey with Swarm 1.2. Setting up Deployment Position for Survey/" + str(robot_number)+ " Robots/"
    R = (solution)
    timer = 1000
    time = 1000

    print(solution)
    print(robot_number)
    print(max_force)
    #setting physics constants
    power = 2
    print(solution)
  
    print(R)

    multiply = 1.5

    #initialising parameters
    conversion_ms = 1/3.6 #km/h to m/s conversion actor
    
    #Robot Parameters
    robot_speed = 2 #m/s
    robot_radius = 1 
    robot_mass = 1 #kgfro
    x_robot = []
    y_robot = []
    min_neighbour = []
    gen_random = 0 #the random angle

    #dust devil parameters
    dust_time = 170 #time in seconds the dust devil will survive 
    dust_speed = 5 #m/s
    dust_radius = 6
    dust_devils = []
    detection = 100

    #Area Parameters
    area = 1 #area in km^2 
    side = 1 #each side length in m
    per_square_km = 100/(60*60*24)
    probability_dust = per_square_km*4#(per_square_km*area) #number of dust devils per second



    lattice_constants = [math.sqrt(3),math.sqrt(2)]
    toggle_probability = 0.5
    G_scale = [0.9,0.95,1,1.05,1.1]
    initialise_array = [20]
    lattice = lattice_constants[0]
    honeycomb = True
    #robot_number = 100#100
    
    first_path = base_path
    try:
        os.mkdir(first_path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % first_path)
    else:
        print ("Success: Directory creation of %s succeeded" % first_path)


    #robot_speed_array = [1,2,3,4,5,6,7,8,9,10]
    
    max_force_overall = 99
    velocity_coefficient_array = [0.5,0.1,0.9]

    path = first_path + str(robot_number) + " Robots/" 
    try:
        os.mkdir(path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % path)
    else:
        print ("Success: Directory creation of %s succeeded" % path)
    image = (first_path + "Images/")
    try:
        os.mkdir(image)
    except OSError:
        print ("Failure: Directory creation of %s failed" % image)
    else:
        print ("Success: Directory creation of %s succeeded" % image)
        

    force_code = "FMAX_" + str(max_force)
    code = str(robot_number) + " Robots"
    

    values = []
    timestep = 1
    frequency = 1

    #R_array = [20,50,100,150,200,250,500,1000]#[10,20,50,100]
    #R_array = [50,60,75,90,100]
    grid = np.array([-500,500])
    grid_length = 10
    grid_performance = []


    velocity_coefficient = velocity_coefficient_array[0]

    G_state = G_transition(max_force,R,power)


    random_seed = seed#random.randint(0,10000)
    random.seed(random_seed)#random_seed)

    performance_metric = []





    #for initial in initial_array:

    #only set seed if eliminating bias
    #random_seed = os.urandom(8)
    #random.seed(random_seed)
    #setting up the robot setup
    swarm = initialise(robot_number, side, initialise_array[0],robot_speed,R,random_seed, lattice_constants)

    store_robots = np.zeros((2,robot_number,(time*timestep)))
    store_dust = np.zeros((2,robot_number,(time*timestep)))
    min_neighbours = []
    cluster_average = []
    store_dust = []
    dust_devils = []
    measurement_events = 0
    detection_metric = 0
    dust_count = 0 
    total_detection = []
    total_collision = []
    total_dust = []

   
    store_robots[:,:,0] = positions(swarm)
    store_dust.append([[[], []]])
    for j in range(0,time*frequency):
        """if(2500<time<4000):
            velocity_coefficient = velocity_coefficient_array[1]
        if(time>4000):
            velocity_coefficient = velocity_coefficient_array[2]"""
        """if(j%100==0 and j>0):
            honeycomb = not honeycomb

        if(honeycomb):
            lattice = lattice_options[0]
        else:
            lattice = lattice_options[1]"""
        print(j)
        update_decay(swarm,R)
        #updating robots and storing robot/dust devil positions

        store_dust.append(positions(dust_devils))

        #checking if 1 second has passed and updating velocities of the robots via the physics walk algorithm and updating status of dust devils
        if(j%frequency==0):
            dust_count = dust_count + dust(dust_devils,1,side,j,dust_speed,dust_time,timestep,frequency,seed)
            physics_walk_honey(swarm,G_state,power,R,max_force,multiply,1,lattice,velocity_coefficient,max_force_overall)    
            
        #calling update timestep method which updates positions of robots, and returns performance metrics
        measurement_events_change,detection_metric_change = update_timestep(swarm,dust_devils,timestep,frequency,min_neighbours,cluster_average,detection,R,multiply,20,5)
        measurement_events = measurement_events + measurement_events_change
        detection_metric = detection_metric + detection_metric_change

        #appending performance metrics into list for storage/post processing
        total_detection.append(detection_metric)
        total_collision.append(measurement_events)

        #updates dust devils positions
        update_dust(dust_devils)

        #storing the number of dust devils generated
        total_dust.append(dust_count)
        store_robots[:,:,j] = positions(swarm)
    type_0_object,type_1_object = split(swarm)
    x_0,y_0 = positions(type_0_object)
    x_1,y_1 = positions(type_1_object)

    print(store_robots[:,0])
    print(store_robots[:,].size)
    current_grid_metric = grid_coverage(grid,store_robots[:,:,time*frequency-1],grid_length)
    #saving robot positions
    np.save(path + "Robots.npy",store_robots)

    #saving dust devil positions
    with open( path + "dust.txt", "w") as f:
        json.dump(store_dust, f)

    #saving performance metrics - these could not be stored in excel due to storage limits
    np.save(path + "Minimum Distance to Neighbours.npy",np.array(min_neighbours))
    np.save(path + "Cluster Average.npy", np.array(cluster_average))
    np.save(path + "Measurement Events Count.npy", np.array(total_collision))
    np.save(path + "Number of Dust Devils Detected.npy", np.array(total_detection))
    np.save(path + "Number of Dust Devils Generated.npy", np.array(total_dust))


    #adding values tested for a values excel spreadsheet for easy viewing
    values.append([len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,min_neighbours[time*frequency-1],cluster_average[time*frequency-1],random_seed, G_state,measurement_events,detection_metric,dust_count])

    #the titles of the variables in the values array
    values_title = ['Number of Robots',
                            'Timestep',
                            'G', 
                            'Power', 
                            'Communication Range', 
                            'Multiplier',
                            'Max Force',
                            'Max Speed',
                            'Minimum Distance to Neighbours',
                            'Average Cluster Size',
                            'Seed',
                            'G Transition',
                            'Measurement Events Count',
                            'Number of Dust Devils Detected',
                            'Total Number of Dust Devils',
        ]

    #storing the constants used in the simulation
    data = {'Values':[len(swarm),timestep,power,R,multiply,max_force,robot_speed,random_seed, G_state]}
    titles =['Number of Robots',
                            'Timestep', 
                            'Power',
                            'Communication Range', 
                            'Multiplier',
                            'Max Force',
                            'Max Speed',
                            'Seed',
                            'G',
        ]
    #setting the constants as a panda dataframe
    constants = pd.DataFrame(data, index = titles
                           ) 

    constants.to_excel(path + "Constants.xlsx")
    print("G_state: ",G_state)
    print("G_parameter: " ,G_state)
    #setting the start paths for the graphs and the tables
    graph_start_path = image + code + "- Graph_Beginning.png"
    table_start_path = image +code + "- Table_Beginning.png"

    code = str(robot_number) + " Robots"
    
    #using the processing functions to create plotly graphs and tables for the figures in the first timestep
    graph_start = Processing_Functions_Tracking.graph_figure(store_robots,0,frequency,code)
    graph_start.write_image(graph_start_path)
    table_start = Processing_Functions_Tracking.table_figure(store_robots,0,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
    table_start.write_image(table_start_path)

    #combining the tables and the graphs using pillow
    Processing_Functions_Tracking.combine(graph_start_path,table_start_path)
    #setting the end paths for the graphs and the tables
    graph_end_path = image + code + "- Graph_End.png"
    table_end_path =image + code + "- Table_End.png"

    #using the processing functions to create plotly graphs and tables for the figures in the last timestep
    graph_end = Processing_Functions_Tracking.graph_figure(store_robots,time-1,frequency,code)
    graph_end.write_image(image + code + "- Graph_End.png")
    table_end = Processing_Functions_Tracking.table_figure(store_robots,timer-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
    table_end.write_image(image + code + "- Table_End.png")

    #using the processing functions to create plotly graphs and tables for the figures in the last timestep
    graph_end_types = Processing_Functions_Tracking.graph_types(x_0,y_0,x_1,y_1,500,10,"Deployed Swarm Formation ", " <b>Timestep = " + str(time) + " s<br>R = " + str(round(R,2)) + "</b> <br> ")
    graph_end_types.write_image(image + code + "- Graph_End_Types.png")


    #using the processing functions to create plotly graphs and tables for the figures in the last timestep
    graph_end_area_coverage = Processing_Functions_Tracking.graph_area_coverage(x_0,y_0,x_1,y_1,500,10, "Area Coverage Over a Grid for a Deployed Swarm Formation"," <b>Timestep = " + str(time) + " s<br>R = " + str(round(R,2)) + "<br>Area Coverage = " + str(current_grid_metric) + "%</b><br> ")
    graph_end_area_coverage.write_image(image + code + "- Graph_End_Area_Coverage.png")
    #table_end_types = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
    #table_end.write_image(image + "/" + code + "- Table_Types_End.png")

    #combining the tables and the graphs using pillow
    Processing_Functions_Tracking.combine(graph_end_path,table_end_path)

    #plotting performance of the average of minimum neighbouring distance metric
    performance = Processing_Functions_Tracking.performance_graph(min_neighbours,np.linspace(0,len(min_neighbours),len(min_neighbours)*frequency,endpoint = False),frequency,code,"Time (s)","Minimum Average Neighbour Distance (m)")
    performance.write_image(image + code + "- Minimum Neighbour Average.png")
    
    #plotting performance of the dust devil measurement metric
    performance_intercept = Processing_Functions_Tracking.performance_graph(total_collision,np.linspace(0,len(total_collision),len(total_collision)*frequency,endpoint = False),frequency,code,"Time (s)","Count of Measurement Events")
    performance_intercept.write_image(image + "/" + code + "- Intercept Performance.png")
    
    detection_code = code + " and " + str(total_dust[time*frequency-1]) +" Dust Devils"
    #plotting performance of the dust devil detection metric
    performance_detection = Processing_Functions_Tracking.performance_graph(total_detection,np.linspace(0,len(total_detection),len(total_detection)*frequency,endpoint = False),frequency,detection_code,"Time (s)","Count of Dust Devils Detected")
    performance_detection.write_image(image + "/" + code + "- Detection Performance.png")
    

    #plotting performance of the cluster average of the swarm
    cluster = Processing_Functions_Tracking.performance_graph(cluster_average,np.linspace(0,len(cluster_average),len(cluster_average)*frequency,endpoint = False),frequency,code,"Time (s)","Average Cluster Size")
    cluster.write_image(image + code +  "- Average Cluster Size.png")




    #setting panda dataframe for the values tested
    df = pd.DataFrame(values, columns = values_title)

    #saving at as an excel spreadsheet
    df.to_excel(path + "Results.xlsx")


    #performance_metric_np = np.array(performance_metric)

    #saving performance metrics
    #np.save(original_path + "Performance Metric " + str(timestep) + ".npy", performance_metric_np)

    #plotting performance of the different timesteps
    #performance_overall = Processing_Functions_BP.performance_graph(performance_metric_np[:,1],performance_metric_np[:,0],1,code,"Number of Timesteps in 1 Second","Number of Dust Devils Detected")
    #performance_overall.write_image(new_path + "Overall Performance.png")
    grid_performance.append([current_grid_metric,robot_number])

    interception_metric.append([robot_number,total_collision[-1]])
    performance_metric.append([robot_number,total_detection[-1]])


    performance_metric_np = np.array(grid_performance)
    np.save(path + code + " Grid Performance Metric.npy", performance_metric_np)
    #performance_overall = Processing_Functions_Tracking.performance_graph(performance_metric_np[:,0],performance_metric_np[:,1],1,code,"Desired Seperation Distance, R (m) ","Grid Coverage Percentage")
    #performance_overall.write_image(original_path + "Overall Performance.png")

    print(100-current_grid_metric)
    x_max = max(abs(store_robots[0,:,time*frequency-1]))
    y_max = max(abs(store_robots[1,:,time*frequency-1]))

   

In [50]:

def optimise(i,base_path):
    from scipy.optimize import brute
    from scipy.optimize import fmin
    from scipy import optimize
    robot_array = [10,20,30,40,50,60,70,80,90,100]
    max_force_array = [25,10,25,10,50,99,10,99,5,10]
    #bound_array = ((200,250),(150,200),(150,200),(100,150),(100,150),(100,150),(100,150),(50,100),(50,100),(50,100),)
    seed_array = [9171,8853,1819,5251,9342,5453,7433,9620,9625,7543]
    R_array = [244.44444444444446,
               166.66666666666666,
               116.66666666666666,
               111.45833333333334,
               113.74999999999999,
               110.41666666666669,
               105.8029513888889,
               94.44444444444444,
               83.33333333333333,
               77.77777777777777]
               
    R=R_array[i]
    robot_number = robot_array[i]
    max_force = max_force_array[i]
    #mybounds = bound_array[i]
    seed = seed_array[i]
    
   
    global performance_metric
    global interception_metric
    global grid_performance
    
 

    
   
    performance_metric = []
    interception_metric = []
    grid_performance = []
    
    robot_number = robot_array[i]
    #mybounds = bound_array[i]
    print("Test")
    args = (robot_number,max_force,seed)

    
    simulation_run(R, robot_number,max_force,seed,base_path)


    
    path = base_path+str(robot_number)+" Robots/"


   
    
   

In [51]:
from scipy.optimize import minimize
global fitness_results
from concurrent import futures
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
from functools import partial
from itertools import zip_longest
from itertools import chain

base_path = "../Experiments/Detecting Dust Devils - Passive Survey with Swarm 1.3. Testing Dust Devil Detection/"

num_cores = multiprocessing.cpu_count()
robot_array = [10,20,30,40,50,60,70,80,90,100]
print(len(robot_array))



global interception_metric
global grid_performance

global performance_metric
global interception_metric


interception_metric = []
grid_performance = []
performance_metric = []


results = Parallel(n_jobs=num_cores)(delayed(optimise)(i,base_path) for i in range(0,10))




10


In [ ]:

i=2
from scipy.optimize import brute
from scipy.optimize import fminbound
from scipy import optimize
robot_array = [10,20,30,40,50,60,70,80,90,100]
max_force_array = [25,10,25,10,50,99,10,99,5,10]
bound_array = ((200,250),(150,200),(150,200),(100,150),(100,150),(100,150),(100,150),(50,100),(50,100),(50,100),)
seed_array = [9171,8853,1819,5251,9342,5453,7433,9620,9625,7543]
#R_array = [100,100,100,100,100,100,100,100,100,100]
#R=R_array[i]
robot_number = robot_array[i]
max_force = max_force_array[i]
mybounds = bound_array[i]
print(mybounds)
seed = seed_array[i]
global max_r_values
global max_performance
global fitness_results
if(robot_number>=50):
    stepsize = 20
else:
    stepsize = 10

base_path = "../Experiments/Area Coverage 2. Optimisation of R for Chosen FMAX Parameters using Scipy Brute/"

fitness_results = []
robot_number = robot_array[i]
mybounds = bound_array[i]
print("Test")
args = (robot_number,max_force,seed)
print(args)
results = brute(simulation_run,((100,150),),Ns=10,args= args, full_output=True, finish=lambda simulation_run, xmin, args=(): fmin(simulation_run, xmin, args, full_output=True, xtol=0.5))
temp = np.array(fitness_results)

results_sorted = temp[temp[:,0].argsort()]
path = base_path+str(robot_number)+" Robots/"


np.save(path + "Area_Coverage_Results.npy",results_sorted)
performance_overall = Processing_Functions_Tracking.performance_graph(results_sorted[:,1],results_sorted[:,0],1,"R" + str(round(results_sorted[-1,0],2)),"Desired Seperation Distance, R (m) ","Grid Coverage Percentage")
performance_overall.write_image(path + "Area Coverage (%) vs Desired Seperation Distance R (m) Performance.png")

maximum = results_sorted[np.where(results_sorted[:,0] == np.amax(results_sorted[:,0]))]
temp_max = maximum[0]
max_performance.append(temp_max)
max_r_values.append(temp_max[0])



In [ ]:
from itertools import chain

for i in chain(range(0,3),range(4,10)):
    print(i)

In [ ]:
print("Success")
global counter
counter = 0
global maximum
global x_previous
global test 
x_previous = 0
global minimum
global count
global fitness_results
maximum = 100
minimum = 150
count = 0
fitness_results = []
global actual_results


    
previous_value = 0
from scipy.optimize import fmin

robot_number = 40 
max_force = 10
seed = 5251
bnds = ((100,150.0),)
args = (robot_number,max_force,seed)
results = fmin(simulation_run, 125,args = args,maxiter = 5,full_output = True, disp = True)


"""kwargs = {"method":"CG","args": (robot_number,max_force,seed),"bounds":bnds }
results = basinhopping(simulation_run, 100,niter = 20,minimizer_kwargs=kwargs, stepsize = 5)"""
print(results)

temp = np.array(fitness_results)

results_sorted = temp[temp[:,0].argsort()]
path = "../Experiments/Area Coverage 2. Optimisation of R for Chosen FMAX Parameters using Scipy/40 Robots/"


np.save(path + "Area_Coverage_Results.npy",results_sorted)
performance_overall = Processing_Functions_Tracking.performance_graph(results_sorted[:,1],results_sorted[:,0],1,"R" + str(results_sorted[-1,0]),"Desired Seperation Distance, R (m) ","Grid Coverage Percentage")
print(max(results_sorted[:,0]))
performance_overall.write_image(path + "Area Coverage (%) vs Desired Seperation Distance R (m) Performance.png")
print(results_sorted)

In [ ]:
results_sorted = np.array([[2,4],[1,6],[8,2]])
maximum = results_sorted[np.where(results_sorted[:,1] == np.amax(results_sorted[:,1]))]
temp = maximum[0]
print("Max: ",temp[1])

In [ ]:

print(len(results_sorted))
print(results.values())
print(results)
print(results.x)
print(results.fun)
result = results_sorted[np.where(results_sorted[:,1] == np.amax(results_sorted[:,1]))]
result

In [ ]:
global previous_value
previous_value = 0
from scipy.optimize import minimize
global robot_number,max_force 
robot_number = 40 
max_force = 10
seed = 5251
bnds = ((100,150),)
data_0 = [10,25]
args = (robot_number,max_force,seed)
cons = (
        {'type':'eq','fun': lambda x : max([x[i]-int(x[i]) for i in range(len(x))])})

global fitness_results
fitness_results = []

    
#entry = {"maxiter":10}
tolerance = (100-robot_number)
result = minimize(simulation_run, 100, bounds = bnds,args=args,constraints = cons,options=entry,tol=tolerance)
path = "../Experiments/Area Coverage 2. Optimisation of R for Chosen FMAX Parameters using Scipy/" + str(robot_number) + " Robots/"
fitness_results_np = np.array(fitness_results])
print(result)
print(fitness_results)
print(fitness_results_np[])
np.save(path + "Area_Coverage_Results.npy",fitness_results_np)
performance_overall = Processing_Functions_Tracking.performance_graph(fitness_results_np[:,0],fitness_results_np[:,1],1,"R" + fitness_results[-1,0],"Desired Seperation Distance, R (m) ","Grid Coverage Percentage")

performance_overall.write_image(path + "Area Coverage (%) vs Desired Seperation Distance R (m) Performance.png",performance_overall)
print(fitness_results)

In [ ]:
x = np.array([[3,0],[1,1]])
print(x[:,1])

In [ ]:
global robot_number
global max_force

robot_number = 40 
max_force = 10

data_0 = [10,25]
ga = pyeasyga.GeneticAlgorithm(data)

ga.fitness_function = simulation_run

In [ ]:
import pygad

global robot_number
global max_force

robot_number = 40 
max_force = 10

num_generations = 10
num_parents_mating = 4

fitness_function = simulation_run

sol_per_pop = 4
num_genes = len(function_inputs)

init_range_low = 100
init_range_high = 150

parent_selection_type = "sss"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

In [ ]:
simulation_run(10)

In [ ]:
from concurrent import futures
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

num_cores = multiprocessing.cpu_count()

robot_array = [100,10,25,50,75,20,30,40,60,70,80,90]
results = Parallel(n_jobs=num_cores)(delayed(simulation_run)(i) for i in robot_array)
print(results)

In [ ]:
print(positions(swarm))
print(store_robots[:,:,time-1])
current_grid_metric = grid_coverage(grid,store_robots[:,:,(time-1)*frequency],grid_length)
current_grid_metric = grid_coverage(grid,store_robots[:,:,time*frequency-1],grid_length)
print(current_grid_metric)

In [ ]:
from matplotlib.pyplot import plot
type_0,type_1 = split(swarm)
x_0,y_0 = np.array(positions(type_0))
x_1,y_1 = np.array(positions(type_1))
print(positions(swarm))

plot(x_0, y_0, 'bo')
plot(x_1, y_1, 'go')

In [ ]:
type_0,type_1 = split(swarm)
x_0,y_0 = np.array(positions(type_0))
x_1,y_1 = np.array(positions(type_1))
#def graph_area_coverage(x_0,y_0,x_1,y_1,maximum,number):
x_overall = np.concatenate((x_0, x_1), axis=None)
y_overall = np.concatenate((y_0,y_1),axis = None)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_0,
    y=y_0,
    mode='markers',
    showlegend=True,
    name="Type 0",
    marker=dict(
        symbol='circle',
        opacity=0.8,
        color='red',
        size=8,
        line=dict(width=1),
    )
))
fig.add_trace(go.Scatter(
    x=x_1,
    y=y_1,
    mode='markers',
    showlegend=True,
     name="Type 1",
    marker=dict(
        symbol='circle',
        opacity=0.8,
        color='cyan',
        size=8,
        line=dict(width=1),
    )
))

fig.add_trace((go.Histogram2d(x=x_overall, y=y_overall,
        autobinx=False,
        xbins=dict(start=-200, end=200, size=40),
        autobiny=False,
        ybins=dict(start=-200, end=200, size=40),
        zmax=1,            
        zauto=False,
        showscale = False,
        colorscale=["white","black"],
        )))
fig.update_xaxes(showgrid=True,showline=True, linewidth=2, linecolor='black',mirror=True)
fig.update_yaxes(showgrid=True,showline=True, linewidth=2, linecolor='black',mirror=True)
fig.update_xaxes(showgrid=True,zeroline=False, gridwidth=1, gridcolor='black')
fig.update_yaxes(showgrid=True,zeroline=False, gridwidth=1, gridcolor='black')
fig.update_traces(opacity=0.4, selector=dict(type='histogram2d'))
fig.update_layout(
#     height=900,width=1150,
     xaxis = dict(
         tickmode = 'linear',
         tick0 = 0,
         dtick = 200/5
     ),
                       yaxis = dict(
         tickmode = 'linear',
         tick0 = 0,
         dtick = 200/5,
        
         
     ))
fig.add_annotation(text='<b>Timestep: 00d00h20s <br> Seperation (R) = 20 <br> Area Coverage = 20% </b>',
                align='center',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=1.14,
                y=0.8,
                bordercolor='black',
                borderwidth=2, 
                  font=dict(

        size=8
        
    ))
fig.show()

In [ ]:
"""original_path = "../Experiments/Initial_Types_Test/"
try:
    os.mkdir(original_path)
except OSError:
    print ("Failure: Directory creation of %s failed" % original_path)
else:
    print ("Success: Directory creation of %s succeeded" % original_path)
    
code = "Run_" + str(initialise_array[i])
path = (original_path + code + "/")
try:
        os.mkdir(path)
except OSError:
    print ("Failure: Directory creation of %s failed" % path)
else:
    print ("Success: Directory creation of %s succeeded" % path)


image = (original_path + "Images/")
try:
    os.mkdir(image)
except OSError:
    print ("Failure: Directory creation of %s failed" % image)
else:
    print ("Success: Directory creation of %s succeeded" % image)

#saving performance metrics - these could not be stored in excel due to storage limits
np.save(path + "Minimum Distance to Neighbours.npy",np.array(min_neighbours))
np.save(path + "Cluster Average.npy", np.array(cluster_average))
np.save(path + "Measurement Events Count.npy", np.array(total_collision))
np.save(path + "Number of Dust Devils Detected.npy", np.array(total_detection))
np.save(path + "Number of Dust Devils Generated.npy", np.array(total_dust))


#adding values tested for a values excel spreadsheet for easy viewing
values.append([len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,min_neighbours[time*frequency-1],cluster_average[time*frequency-1],random_seed, G_state,measurement_events,detection_metric,dust_count])

#the titles of the variables in the values array
values_title = ['Number of Robots',
                        'Timestep',
                        'G', 
                        'Power', 
                        'Communication Range', 
                        'Multiplier',
                        'Max Force',
                        'Max Speed',
                        'Minimum Distance to Neighbours',
                        'Average Cluster Size',
                        'Seed',
                        'G Transition',
                        'Measurement Events Count',
                        'Number of Dust Devils Detected',
                        'Total Number of Dust Devils',
    ]

#storing the constants used in the simulation
data = {'Values':[len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,random_seed, G_state]}
titles =['Number of Robots',
                        'G',
                        'Timestep', 
                        'Power',
                        'Communication Range', 
                        'Multiplier',
                        'Max Force',
                        'Max Speed',
                        'Seed',
                        'G Transition',
    ]
#setting the constants as a panda dataframe
constants = pd.DataFrame(data, index = titles
                       ) 

constants.to_excel(path + "Constants.xlsx")

#setting the start paths for the graphs and the tables
graph_start_path = image + code + "- Graph_Beginning.png"
table_start_path = image + "/" + code + "- Table_Beginning.png"

#using the processing functions to create plotly graphs and tables for the figures in the first timestep
graph_start = Processing_Functions_Tracking.graph_figure(store_robots,0,frequency,code)
graph_start.write_image(graph_start_path)
table_start = Processing_Functions_Tracking.table_figure(store_robots,0,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
table_start.write_image(table_start_path)

#combining the tables and the graphs using pillow
Processing_Functions_Tracking.combine(graph_start_path,table_start_path)

#setting the end paths for the graphs and the tables
graph_end_path = image + code + "- Graph_End.png"
table_end_path = image + "/" + code + "- Table_End.png"

#using the processing functions to create plotly graphs and tables for the figures in the last timestep
graph_end = Processing_Functions_Tracking.graph_figure(store_robots,time-1,frequency,code)
graph_end.write_image(image + "/" + code + "- Graph_End.png")
table_end = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
table_end.write_image(image + "/" + code + "- Table_End.png")

#combining the tables and the graphs using pillow
Processing_Functions_Tracking.combine(graph_end_path,table_end_path)

#plotting performance of the average of minimum neighbouring distance metric
performance = Processing_Functions_Tracking.performance_graph(min_neighbours,np.linspace(0,len(min_neighbours),len(min_neighbours)*frequency,endpoint = False),frequency,code,"Time (s)","Minimum Average Neighbour Distance (m)")
performance.write_image(image + "/" + code + "- Performance.png")

#plotting performance of the dust devil measurement metric
performance_intercept = Processing_Functions_Tracking.performance_graph(total_collision,np.linspace(0,len(total_collision),len(total_collision)*frequency,endpoint = False),frequency,code,"Time (s)","Count of Measurement Events")
performance_intercept.write_image(image + "/" + code + "- Intercept Performance.png")

#plotting performance of the cluster average of the swarm
cluster = Processing_Functions_Tracking.performance_graph(cluster_average,np.linspace(0,len(cluster_average),len(cluster_average)*frequency,endpoint = False),frequency,code,"Time (s)","Average Cluster Size")
cluster.write_image(image + "/" + code + "- Average Cluster Size.png")




#setting panda dataframe for the values tested
df = pd.DataFrame(values, columns = values_title)

#saving at as an excel spreadsheet
df.to_excel(path + "Results.xlsx")


#performance_metric_np = np.array(performance_metric)

#saving performance metrics
#np.save(original_path + "Performance Metric " + str(timestep) + ".npy", performance_metric_np)

#plotting performance of the different timesteps
#performance_overall = Processing_Functions_BP.performance_graph(performance_metric_np[:,1],performance_metric_np[:,0],1,code,"Number of Timesteps in 1 Second","Number of Dust Devils Detected")
#performance_overall.write_image(new_path + "Overall Performance.png")
"""

In [94]:
robot_number = 100
path = "../Experiments/Detecting Dust Devils - Passive Survey with Swarm 1.1. Testing the graphing capabilities and if Dust Devils are Showing Up/" + str(robot_number)+ " Robots/"
store_robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
     store_dust = json.load(f)

constants = pd.read_excel(path + "Constants.xlsx", index_col=0)
min_neighbours = np.load(path + "Minimum Distance to Neighbours.npy")
cluster_average = np.load(path + "Cluster Average.npy")
total_collision = np.load(path + "Measurement Events Count.npy")
total_detection = np.load(path + "Number of Dust Devils Detected.npy")
total_dust = np.load(path + "Number of Dust Devils Generated.npy")

In [95]:
#Function to check if any of the items contain an array
def array_check(item,timer):
    item = item[0]
    print(item)
    if(isinstance(item, str)):
        print(item)
        item = literal_eval(item)
        return round(item[timer],2)
    elif(isinstance(item, list)):

        return round(item[-1],2)
    else:

        return round(float(item),2)

In [96]:
#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [101]:
#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer
    if(timer>99):
        timer = 0

    position_robot = store_robots[:,:,timer]
    position_dust = store_dust[timer]
    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-1000, 1000]),xaxis = dict(range=[-1000,1000]))
        }
    )

    fig.add_trace(go.Scatter(
          x= position_dust[0],
          y= position_dust[1],
          name='Dust Devil',
          mode = 'markers',
          marker_color='rgba(255, 182, 193, .9)',
      ))
    
    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Physics Based Swarm Experiment </b>",
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    timer = timer + 1
    
    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    fig = Processing_Functions_Tracking.table_figure(store_robots,timer,1,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
                
    fig.update_layout(width = 650, height = 800,margin=dict(
    t=200, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.2')

Dash is running on http://127.0.0.2:8050/

Dash is running on http://127.0.0.2:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.2:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Sep/2021 01:55:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:14] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:14] "GET /_favicon.ico?v=1.17.0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:15] "POST /_dash-update-component H

127.0.0.1 - - [02/Sep/2021 01:55:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 01:55:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.

...

127.0.0.1 - - [02/Sep/2021 02:11:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Sep/2021 02:11:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:11:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [02/Sep/2021 02:12:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:12:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [02/Sep/2021 02:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\franc\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\franc\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\franc\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\franc\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\franc\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\franc\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\franc\anaconda3\lib\site-packages\dash\dash.py", line 1076, in dispatch
    response.set_d

127.0.0.1 - - [02/Sep/2021 02:13:57] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Sep/2021 02:13:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:13:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 02:14:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [ ]:
def R_return(swarm):
    R_array = []
    for i in swarm:
        R_array.append(i.R)
    return R_array

In [ ]:
print(R_return(swarm))
print(positions(swarm))

In [ ]:
R=20
x_0 =np.array([  0,  -6,   9,   0, -10,   5,  -5, -13, -10,   0, -25,  10,  -1,
         4,  -5,   6, -21, -24, -18,   2, -19,  -9, -11,  -4,  10,   1,
         3, -14,   2, -11, -18, -12,   6,  -7, -20, -27, -20,  -6, -13,
        -9,  12,   7, -21,   8, -14, -11, -14,   8,  11, -14])
y_0 = np.array([ 0, 10,  3,  2, 19, 27, -3, 25, 29, 35, 31, 17, 26, 35, 21, 32, 29,
       36, 34, -4, 19,  7, 17, -3, -4, -2, 30, 25, 17, 15, 33, 21, 34,  0,
        1, 14, 36, 14, 20, 19, 21, 22, 21, -1, -3, 10, 35,  5,  6, 18])

#calculating the distance from the robots to the current robot, split up according to robot type
distance_0 = np.sqrt(np.square(x_0)+np.square(y_0))

print(distance_0)

#finding the corresponding booleans
multiply_like = 1.3
multiply_unlike = 1.7
constant = 0

distance_local_0 = np.logical_and(distance_0>0,distance_0<multiply_like*R)

print(distance_local_0)
print(distance_0[distance_local_0])




x_dist_0 = x_0[distance_local_0]
y_dist_0 = y_0[distance_local_0]
print("X Positions in Neighbourhood: ", x_dist_0)
print("Y Positions in Neighbourhood: ", y_dist_0)
positions_0 = np.array([x_dist_0,y_dist_0])
print(np.sqrt(np.square(positions_0[0])+np.square(positions_0[1])))
for x,y in zip(positions_0[0],positions_0[1]):
    print(magnitude(x,y))
    current_position = np.array([x,y])


    #calculating magnitude of positions
    mag_unshifted = (magnitude(current_position[0],current_position[1]))
    print(mag_unshifted)

In [ ]:
#function to calculate area coverage using grid metric
def grid_coverage(grid,positions,length):
    #creating the grid bins for area coverage
    x_bins = np.linspace(grid[0],grid[1],length+1)
    y_bins = np.linspace(grid[0],grid[1],length+1)
    
    x_positions = positions[0]
    y_positions = positions[1]
    #initialising empty coordinates array for use in the grid metric
    coordinates = []
    for i in range(len(x_positions)):
        #setting current position
        x_current = x_positions[i]
        y_current = y_positions[i]
        #searching for the x anf y coordinates using the numpy search sorted function combined with the previously defined bins
        x = np.searchsorted(x_bins, x_current)
        y = np.searchsorted(y_bins, y_current)
        if((0<=x<length+1) and (0<=y<length+1)):
            coordinates.append((x,y))
        print(coordinates)


    rm_duplicates = list(set(coordinates)) #set is used to remove duplicates, as sets require distinct elements
    area_coverage = len(rm_duplicates)*0.01 #converting it to a percentage
    total = length*length
    return area_coverage

In [ ]:
x_0 =np.array([  0,  -6,   9,   0, -10,   5,  -5, -13, -10,   0, -25,  10,  -1,
         4,  -5,   6, -21, -24, -18,   2, -19,  -9, -11,  -4,  10,   1,
         3, -14,   2, -11, -18, -12,   6,  -7, -20, -27, -20,  -6, -13,
        -9,  12,   7, -21,   8, -14, -11, -14,   8,  11, -14])
y_0 = np.array([ 0, 10,  3,  2, 19, 27, -3, 25, 29, 35, 31, 17, 26, 35, 21, 32, 29,
       36, 34, -4, 19,  7, 17, -3, -4, -2, 30, 25, 17, 15, 33, 21, 34,  0,
        1, 14, 36, 14, 20, 19, 21, 22, 21, -1, -3, 10, 35,  5,  6, 18])
(grid_coverage(np.array([-500,500]),np.array([x_0,y_0]),100))

In [ ]:
grid = [-500,500]
x = np.linspace(-500,500,11)
y = np.linspace(-500,500,11)
print(x)
print(y)

position_array = np.array([[-6000,6000],[-6000,6000]]) #store_robots[:,:,1999]
print("Positions:",position_array)
print("Independent review: ",position_array[0])
coordinates = []
for i in range(len(position_array[0])):
    position = (position_array[:,i])
    coordinates.append(((np.searchsorted(x, position[0])),(np.searchsorted(y, position[1]))))
    print(coordinates)


rm_duplicates = list(set(coordinates))
print(rm_duplicates)
print(len(coordinates))
print(len(rm_duplicates))

In [ ]:

x_0 =np.array([  0,  -6,   9,   0, -10,   5,  -5, -13, -10,   0, -25,  10,  -1,
         4,  -5,   6, -21, -24, -18,   2, -19,  -9, -11,  -4,  10,   1,
         3, -14,   2, -11, -18, -12,   6,  -7, -20, -27, -20,  -6, -13,
        -9,  12,   7, -21,   8, -14, -11, -14,   8,  11, -14])
y_0 = np.array([ 0, 10,  3,  2, 19, 27, -3, 25, 29, 35, 31, 17, 26, 35, 21, 32, 29,
       36, 34, -4, 19,  7, 17, -3, -4, -2, 30, 25, 17, 15, 33, 21, 34,  0,
        1, 14, 36, 14, 20, 19, 21, 22, 21, -1, -3, 10, 35,  5,  6, 18])
fig = go.Figure(go.Histogram2d(x=x_0, y=y_0,
        autobinx=False,
        xbins=dict(start=-500, end=500, size=50),
        autobiny=False,
        ybins=dict(start=-500, end=500, size=50),
        zmax=1,            
        zauto=False,
        showscale = False
        ))
fig.add_trace(go.Scatter(
    x=x_0,
    y=y_0,
    mode='markers',
    showlegend=False,
    marker=dict(
        symbol='circle',
        opacity=0.7,
        color='white',
        size=8,
        line=dict(width=1),
    )
))
fig.show()

In [ ]:
#Function to call physics based area coverage algorithm
def physics_walk_tracking(swarm,G,power,R,max_force,multiply,timestep):
    '''
    Updates the robots velocities based on a physicomimetics artificial gravity algorithm

            Parameters:
                   Swarm (list): a list of robot objects
                   G (float): the chosen G constant float value
                   power (float): the chosen power for the artificial gravity equation
                   R (float): the chosen R seperation float value
                   max_force (float): the maximum float limit on the force
                   multiply (float): the range multiplication float
            Returns:
                   None

    '''
    #setting initial constants
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    
    #retrieving x and y positions
    x,y = positions(swarm)
    
    #looping through swarm with each individual robot
    for i in range(len(swarm)):
        
        #selecting robots one by one
        robot = swarm[i]
        #print("Robot Position: ", [robot.x,robot.y])
        
        x_copy,y_copy = x.copy(),y.copy()
        
        #initialising the forces as 0
        force = np.array([0,0])

        #finding the distance between the current robot and the other robots
        x_dist_unsorted = np.array(x_copy)-robot.x
        y_dist_unsorted = np.array(y_copy)-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))

        #print("Distance: ",distance)
        #print("Non Zero Distance:", np.nonzero(distance))
        #determining the robots within 1.5R distance
        distance_local = np.logical_and(distance>0,distance<multiply*R)
        neighbourhood = np.where(distance_local)
        list_neighbours = list(neighbourhood[0])
        #print("Indices of local: ",list_neighbours)
        #print(len(np.where(distance_local)))
        #print("Booleans Local: ",distance_local)
        #print(type(distance_local))
        #creating the x and y distance matrices for robots within the local 1.5R distance
        neighbours = [swarm[i] for i in list_neighbours]
        #print("Neighbours:",positions(neighbours))
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        
        #initialising force changes
        force_change_dir = 0
        
        #number of robots within the neighbourhood
        robot_number = len(distance_local)
        

        #looping through the current robots neighbourhood
        for j in range(len(neighbours)):
            
            neighbour = neighbours[j]
            
            #storing current position 
            current_position = np.array([position[0,j],position[1,j]])
                

            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])


            #calculating the force
            numerator = (G*(robot.mass*neighbour.mass))
            denominator = mag**2


            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)

            #calculating the unit vector of the position
            distance_unit = unit(current_position)

            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>robot.R):
                force_change_dir = distance_unit

            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<robot.R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])

            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if((force_delta[0]**2+force_delta[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force_delta[0],force_delta[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force_delta[0] = updating_force[0]
                force_delta[1] = updating_force[1]

            #calculating new force
            force = force+force_delta
            

        #calculating the change in velocity
        delta_vx = (force[0])*timestep/robot.mass
        delta_vy = (force[1])*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy

       #keeping the velocity within the maximum velocity of the robot
        if((velocity_x**2+velocity_y**2)>((robot.max_velocity)**2)):
            #calculating unit vector of the velocity
            unit_velocity = unit([velocity_x,velocity_y])
            
            #multiplying it by the maximum velocity
            velocity = unit_velocity*robot.max_velocity
            
            #setting the new velocity equal to the respective updated maximum velocity components
            velocity_x = velocity[0]
            velocity_y = velocity[1]

        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    
